In [3]:
%cd /home/soda/rcappuzz/work/study-data-lakes/

/home/soda/rcappuzz/work/study-data-lakes


In [71]:
%load_ext autoreload
%autoreload 2
from src import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import polars as pl
from pathlib import Path
import numpy as np

In [8]:
src_path = Path("data/yago3-dl")

In [9]:
tab_paths = list(src_path.iterdir())
first_tab = pl.read_parquet(tab_paths[0])

In [10]:
tabs = {}
for tpath in tab_paths:
    tname = tpath.stem
    tab = pl.read_parquet(tpath)
    tabs[tname] = tab


In [11]:
list(tabs.keys())

['yago_typetab_<wordnet_movie_106613686>',
 'yago_typetab_<wordnet_person_100007846>',
 'yago_typetab_<wordnet_album_106591815>',
 'yago_typetab_<wordnet_administrative_district_108491826>',
 'yago_typetab_<wordnet_artist_109812338>',
 'yago_typetab_<wordnet_company_108058098>',
 'yago_typetab_<wordnet_event_100029378>',
 'yago_typetab_<wordnet_officeholder_110371450>',
 'yago_typetab_<wordnet_season_115239579>',
 'yago_typetab_<wikicat_Living_people>']

In [13]:
utils.profile_dataframe("asd")

TypeError: Table asd has inappropriate type <class 'str'>

In [50]:
%load_ext autoreload
%autoreload 2
from src import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
utils.profile_dataframe(first_tab)

,type,subject,<isLocatedIn>,<hasWebsite>
count_unique,1.0,151678.0,391.000000,17.000000
count_nans,0.0,0.0,75562.000000,154260.000000
nan_frac,0.0,0.0,0.489778,0.999883


In [23]:
for tab_name, tab in tabs.items():
    display(utils.profile_dataframe(tab))

,type,subject,<isLocatedIn>,<hasWebsite>
count_unique,1.0,151678.0,391.000000,17.000000
count_nans,0.0,0.0,75562.000000,154260.000000
nan_frac,0.0,0.0,0.489778,0.999883


,type,subject,<hasGender>,<directed>,<isAffiliatedTo>,<actedIn>,<hasWebsite>,<playsFor>,<isMarriedTo>,<wroteMusicFor>,<hasWonPrize>,<diedIn>,<created>,<wasBornIn>,<isCitizenOf>,<graduatedFrom>
count_unique,1.0,2034731.0,3.000000e+00,1.091410e+05,4.272200e+04,1.163820e+05,8.062200e+04,2.648900e+04,7.214000e+04,6.808400e+04,7.600000e+03,5.140700e+04,3.496710e+05,1.092860e+05,621.000000,9.291000e+03
count_nans,0.0,0.0,1.043073e+06,3.038777e+07,4.442625e+07,2.625245e+07,4.141776e+07,3.654460e+07,2.673404e+07,4.091380e+07,3.155317e+07,3.723250e+07,2.037693e+07,2.489295e+06,363030.000000,5.637327e+07
nan_frac,0.0,0.0,1.624533e-02,4.732740e-01,6.919162e-01,4.088685e-01,6.450604e-01,5.691634e-01,4.163690e-01,6.372115e-01,4.914245e-01,5.798772e-01,3.173603e-01,3.876950e-02,0.005654,8.779848e-01


,type,subject,<isLocatedIn>,<hasWebsite>
count_unique,1.0,156553.0,74.000000,67.000000
count_nans,0.0,0.0,153903.000000,156617.000000
nan_frac,0.0,0.0,0.982251,0.999572


,type,subject,<hasWebsite>,<hasWonPrize>,<isLocatedIn>
count_unique,1.0,470161.0,7.943800e+04,2.210000e+02,51750.000000
count_nans,0.0,0.0,1.270392e+06,1.509896e+06,14246.000000
nan_frac,0.0,0.0,8.408459e-01,9.993686e-01,0.009429


,type,subject,<wroteMusicFor>,<created>,<wasBornIn>,<actedIn>,<diedIn>,<directed>,<graduatedFrom>,<isAffiliatedTo>,<isCitizenOf>,<playsFor>,<hasWonPrize>,<hasGender>,<hasWebsite>,<isMarriedTo>
count_unique,1.0,140435.0,3.787000e+04,176108.00000,18620.000000,6.090000e+03,6.507000e+03,1.081000e+03,9.920000e+02,2.370000e+02,298.000000,2.210000e+03,1024.000000,3.000000,4.093700e+04,3.681000e+03
count_nans,0.0,0.0,3.441457e+06,137953.00000,550665.000000,3.078676e+06,9.965874e+06,6.468565e+06,9.032439e+06,1.116658e+07,50565.000000,7.766856e+06,801960.000000,22662.000000,5.286356e+06,3.338656e+06
nan_frac,0.0,0.0,2.926146e-01,0.01173,0.046821,2.617686e-01,8.473621e-01,5.499986e-01,7.679955e-01,9.494540e-01,0.004299,6.603876e-01,0.068188,0.001927,4.494797e-01,2.838738e-01


,type,subject,<isLocatedIn>,<created>,<isAffiliatedTo>,<hasWonPrize>,<hasWebsite>
count_unique,1.0,81358.0,13175.000000,27263.000000,48.00000,109.000000,60358.000000
count_nans,0.0,0.0,34779.000000,111423.000000,178752.00000,172522.000000,43092.000000
nan_frac,0.0,0.0,0.194108,0.621874,0.99765,0.962879,0.240505


,type,subject,<isLocatedIn>,<hasWebsite>
count_unique,1.0,79735.0,7140.000000,8391.000000
count_nans,0.0,0.0,46061.000000,78670.000000
nan_frac,0.0,0.0,0.462033,0.789131


,type,subject,<wasBornIn>,<isCitizenOf>,<hasGender>,<playsFor>,<directed>,<created>,<isMarriedTo>,<hasWonPrize>,<actedIn>,<isAffiliatedTo>,<wroteMusicFor>,<diedIn>,<hasWebsite>,<graduatedFrom>
count_unique,1.0,169553.0,37973.000000,364.000000,3.000000,2501.000000,629.00000,6524.000000,10798.000000,1441.000000,5804.000000,6183.000000,296.00000,1.376000e+04,9.409000e+03,6.414000e+03
count_nans,0.0,0.0,60977.000000,25371.000000,2699.000000,677811.000000,352035.00000,298032.000000,286011.000000,278792.000000,328615.000000,215797.000000,586809.00000,1.078152e+06,1.108372e+06,1.081162e+06
nan_frac,0.0,0.0,0.048543,0.020197,0.002149,0.539596,0.28025,0.237259,0.227689,0.221942,0.261606,0.171793,0.46715,8.583015e-01,8.823592e-01,8.606977e-01


,type,subject,<hasWebsite>,<isLocatedIn>
count_unique,1.0,106211.0,721.000000,438.000000
count_nans,0.0,0.0,104033.000000,103088.000000
nan_frac,0.0,0.0,0.967812,0.959021


,type,subject,<hasWonPrize>,<wroteMusicFor>,<isCitizenOf>,<hasGender>,<directed>,<isAffiliatedTo>,<graduatedFrom>,<hasWebsite>,<wasBornIn>,<actedIn>,<playsFor>,<created>,<isMarriedTo>,<diedIn>
count_unique,1.0,1034138.0,5.908000e+03,3.393000e+04,312.000000,3.000000,4.846000e+04,3.809900e+04,8.378000e+03,7.149300e+04,6.205600e+04,7.387700e+04,2.527000e+04,2.105170e+05,3.022500e+04,4.820000e+02
count_nans,0.0,0.0,1.896755e+07,2.467162e+07,169476.000000,952241.000000,2.437510e+07,1.686414e+07,2.976189e+07,2.781419e+07,1.399298e+06,2.080206e+07,1.476189e+07,1.770715e+07,2.221214e+07,3.468050e+07
nan_frac,0.0,0.0,5.466642e-01,7.110614e-01,0.004884,0.027445,7.025154e-01,4.860418e-01,8.577682e-01,8.016334e-01,4.032921e-02,5.995368e-01,4.254529e-01,5.103382e-01,6.401766e-01,9.995277e-01


In [78]:
for tab_name, tab in tabs.items():
    print(tab_name)
    display(utils.profile_dataframe(tab))

yago_typetab_<wordnet_movie_106613686>


,type,subject,<isLocatedIn>,<hasWebsite>
count_unique,1.0,151678.0,391.000000,17.000000
count_nans,0.0,0.0,75562.000000,154260.000000
nan_frac,0.0,0.0,0.489778,0.999883


yago_typetab_<wordnet_person_100007846>


,type,subject,<hasGender>,<directed>,<isAffiliatedTo>,<actedIn>,<hasWebsite>,<playsFor>,<isMarriedTo>,<wroteMusicFor>,<hasWonPrize>,<diedIn>,<created>,<wasBornIn>,<isCitizenOf>,<graduatedFrom>
count_unique,1.0,2034731.0,3.000000e+00,1.091410e+05,4.272200e+04,1.163820e+05,8.062200e+04,2.648900e+04,7.214000e+04,6.808400e+04,7.600000e+03,5.140700e+04,3.496710e+05,1.092860e+05,621.000000,9.291000e+03
count_nans,0.0,0.0,1.043073e+06,3.038777e+07,4.442625e+07,2.625245e+07,4.141776e+07,3.654460e+07,2.673404e+07,4.091380e+07,3.155317e+07,3.723250e+07,2.037693e+07,2.489295e+06,363030.000000,5.637327e+07
nan_frac,0.0,0.0,1.624533e-02,4.732740e-01,6.919162e-01,4.088685e-01,6.450604e-01,5.691634e-01,4.163690e-01,6.372115e-01,4.914245e-01,5.798772e-01,3.173603e-01,3.876950e-02,0.005654,8.779848e-01


yago_typetab_<wordnet_album_106591815>


...

,type,subject,<wasBornIn>,<isCitizenOf>,<hasGender>,<playsFor>,<directed>,<created>,<isMarriedTo>,<hasWonPrize>,<actedIn>,<isAffiliatedTo>,<wroteMusicFor>,<diedIn>,<hasWebsite>,<graduatedFrom>
count_unique,1.0,169553.0,37973.000000,364.000000,3.000000,2501.000000,629.00000,6524.000000,10798.000000,1441.000000,5804.000000,6183.000000,296.00000,1.376000e+04,9.409000e+03,6.414000e+03
count_nans,0.0,0.0,60977.000000,25371.000000,2699.000000,677811.000000,352035.00000,298032.000000,286011.000000,278792.000000,328615.000000,215797.000000,586809.00000,1.078152e+06,1.108372e+06,1.081162e+06
nan_frac,0.0,0.0,0.048543,0.020197,0.002149,0.539596,0.28025,0.237259,0.227689,0.221942,0.261606,0.171793,0.46715,8.583015e-01,8.823592e-01,8.606977e-01


yago_typetab_<wordnet_season_115239579>


,type,subject,<hasWebsite>,<isLocatedIn>
count_unique,1.0,106211.0,721.000000,438.000000
count_nans,0.0,0.0,104033.000000,103088.000000
nan_frac,0.0,0.0,0.967812,0.959021


yago_typetab_<wikicat_Living_people>


,type,subject,<hasWonPrize>,<wroteMusicFor>,<isCitizenOf>,<hasGender>,<directed>,<isAffiliatedTo>,<graduatedFrom>,<hasWebsite>,<wasBornIn>,<actedIn>,<playsFor>,<created>,<isMarriedTo>,<diedIn>
count_unique,1.0,1034138.0,5.908000e+03,3.393000e+04,312.000000,3.000000,4.846000e+04,3.809900e+04,8.378000e+03,7.149300e+04,6.205600e+04,7.387700e+04,2.527000e+04,2.105170e+05,3.022500e+04,4.820000e+02
count_nans,0.0,0.0,1.896755e+07,2.467162e+07,169476.000000,952241.000000,2.437510e+07,1.686414e+07,2.976189e+07,2.781419e+07,1.399298e+06,2.080206e+07,1.476189e+07,1.770715e+07,2.221214e+07,3.468050e+07
nan_frac,0.0,0.0,5.466642e-01,7.110614e-01,0.004884,0.027445,7.025154e-01,4.860418e-01,8.577682e-01,8.016334e-01,4.032921e-02,5.995368e-01,4.254529e-01,5.103382e-01,6.401766e-01,9.995277e-01


In [24]:
big_tab = tabs["yago_typetab_<wordnet_person_100007846>"]

In [35]:
first_tab.lazy().select(
    [
        pl.all().map(pl.Series.n_unique).alias("unique_count"),
        pl.all().map(pl.Series.null_count).alias("null_count"),
    ]
).collect()

DuplicateError: column with name 'unique_count' has more than one occurrences

In [59]:
df = pl.DataFrame(
    {"a": [1, 2, 3], "b": ["hello", "foo", "bar"], "c": [1, 1, 1]}
)
df.select([pl.all().sum().suffix("_sum"), pl.all().first().suffix("_first")])

a_sum,b_sum,c_sum,a_first,b_first,c_first
i64,str,i64,i64,str,i64
6,null,3,1,"""hello""",1


In [41]:
utils.profile_polars(df)

,a,b,c
count_unique,3.0,3.0,1.0
count_nans,0.0,0.0,0.0
nan_frac,0.0,0.0,0.0


In [60]:
df = pl.DataFrame(
    {"a": [1, 2, 3], "b": ["hello", "foo", "bar"], "c": [1, 1, 1]}
)

def col_operations(column: pl.Series):
    col_stats = {
                "count_unique": column.n_unique(),
                "count_nans": column.null_count(),
                "nan_frac": column.null_count()/len(column)        
                 }
    
    return col_stats

def profile(df: pl.DataFrame):
    profiling_dict = {}
    for col in df.columns:
        profiling_dict[col] = col_operations(df[col])
    
    return pd.DataFrame(profiling_dict)

profile(df)

,a,b,c
count_unique,3.0,3.0,1.0
count_nans,0.0,0.0,0.0
nan_frac,0.0,0.0,0.0


In [62]:
df.select(
    [
        pl.all().apply(profile)
    ]
)

ComputeError: AttributeError: 'int' object has no attribute 'columns'

In [68]:
df

a,b,c
i64,str,i64
1,"""hello""",1
2,"""foo""",1
3,"""bar""",1


In [74]:
dfp = pd.DataFrame(
    {"a": [1, 2, 3], "b": ["hello", "foo", "bar"], "c": [1, 1, 1]}
)

dfp.apply(utils.col_operations_pd, axis=0)

,a,b,c
count_unique,3.0,3.0,1.0
count_nans,0.0,0.0,0.0
nan_frac,0.0,0.0,0.0


In [73]:
df.apply(col_operations, axis=1)

TypeError: DataFrame.apply() got an unexpected keyword argument 'axis'

In [77]:
dfp = pd.DataFrame(
    {"a": [1, 2, 3], "b": ["hello", "foo", "bar"], "c": [1, 1, 1]}
)

def col_operations(column: pd.Series):
    col_stats = {
                "count_unique": column.nunique(),
                "count_nans": column.isna().sum(),
                "nan_frac": column.isna().sum()/len(column)        
                 }
    
    return pd.Series(col_stats)



dfp.apply(col_operations, axis=0)

,a,b,c
count_unique,3.0,2.000000,1.0
count_nans,0.0,1.000000,0.0
nan_frac,0.0,0.333333,0.0
